# Filter in time domain
Different approach. Take the 128 bin frequency domain response. Convert into time domain to get the filter `kernel`. Use as FIR filter coefficients.

In [ ]:
%pylab notebook
from PIL import Image
import soundfile as sf
import pygame
from time import sleep

In [ ]:
xOffs = 50
img = Image.open("./220px-Lenna_(test_image).png").convert("L")
img = img.crop([xOffs, 0, xOffs+128, img.size[1]])
print(img.size)
imgDat = asarray(img)
img

In [ ]:
# Get desired frequency response
#h_f = zeros(128)
#h_f[0:5] = 1
#h_f[20:22] = 1
#h_f[40:60] = 0.8
h_f = imgDat[10, :].astype("float")

# Calculate FIR coefficients
h_t = fft.irfft(h_f)
h_t = roll(h_t, h_t.size//2)

# Make filter faster by trunctation
trunc = 64
h_t = h_t[127-trunc:127+trunc]
print(h_t.size)

# Make filter smoother by windowing (stop band attenuation!!)
h_t *= hamming(h_t.size)

# Calculate actual response and plot
close("all")
fig, axs = subplots(3,1, figsize=(6,7))
h_ff = fft.rfft(h_t, 2**16)
axs[0].plot(linspace(0, 1, h_f.size), abs(h_f), "o", label="Desired response")
axs[0].plot(linspace(0, 1, h_ff.size), abs(h_ff), label="Actual response")
axs[0].set_xlabel("Normalized frequency")
axs[0].set_ylabel("Magnitude")
axs[1].plot(linspace(0, 1, h_f.size), angle(h_f), "o", label="Desired response")
axs[1].plot(linspace(0, 1, h_ff.size), angle(h_ff), label="Actual response")
axs[1].set_xlabel("Normalized frequency")
axs[1].set_ylabel("Phase")
axs[2].stem(h_t, label="FIR coefficients")
axs[2].set_xlabel("Samples")
for ax in axs:
    ax.legend(loc="upper right")
fig.tight_layout()

# Realtime FIR filter
basically
```python
aRes = convolve(aDat, h_t)
```

In [ ]:
class FIR(object):
    def __init__(self, coeffs):
        self.coeffs = coeffs
        self.X = zeros_like(coeffs)
        
    def filt(self, xIn):
        self.X = roll(self.X, 1)
        self.X[0] = xIn
        return mean(self.X * self.coeffs)

In [ ]:
pygame.mixer.pre_init(sRate, size=-16, channels=1)
pygame.mixer.init()

In [ ]:
aDat, sRate = sf.read("./11_-_Vivaldi_Winter_mvt_2_Largo_-_John_Harrison_violin.ogg", always_2d=True)
aDat = mean(aDat, 1)
print(sRate, aDat.shape)

In [ ]:
aDat = rand(int(1e6))*2 - 1

In [ ]:
secondsPerScan = 6
samplesPerScan = sRate * secondsPerScan
samplesPerLine = int(samplesPerScan / imgDat.shape[0])
print("Linerate:", sRate / samplesPerLine, "Hz")

In [ ]:
f = FIR(ones(128))
res = zeros_like(aDat)
filtInd = 0
for i, s in enumerate(aDat):
    if (i%samplesPerLine) == 0:
        row  = imgDat[filtInd,:].astype("float")
        row -= min(row)
        row /= max(row)
        h_t = fft.irfft(row)
        h_t = roll(h_t, h_t.size//2)
        trunc = 64
        h_t = h_t[127-trunc:127+trunc]
        h_t *= hamming(h_t.size)
        f.coeffs = h_t / sum(h_t)
        filtInd += 1
        if filtInd >= imgDat.shape[0]:
            print("*", end="", flush=True)
            filtInd = 0
    res[i] = f.filt(s)

In [ ]:
res /= amax(res)

In [ ]:
c.stop()

In [ ]:
c = pygame.sndarray.make_sound((res * (2**15)).astype(int16)).play()